In [ ]:
!pip install einx transformers soundfile huggingface_hub

In [ ]:
import sys
import torch
import re
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
import soundfile as sf

# 1. Clone Spark-TTS repo
!git clone https://github.com/SparkAudio/Spark-TTS
sys.path.append('Spark-TTS')
from sparktts.models.audio_tokenizer import BiCodecTokenizer

# 2. Load LLM từ Hugging Face
model = AutoModelForCausalLM.from_pretrained("huyvux3005/spark-tts-vietnamese-5hourr_16bit")
tokenizer = AutoTokenizer.from_pretrained("huyvux3005/spark-tts-vietnamese-5hourr_16bit")

# 3. Load BiCodec (cần tải Spark-TTS-0.5B base model)
from huggingface_hub import snapshot_download
snapshot_download("unsloth/Spark-TTS-0.5B", local_dir="Spark-TTS-0.5B")
audio_tokenizer = BiCodecTokenizer("Spark-TTS-0.5B", "cuda")

fatal: destination path 'Spark-TTS' already exists and is not an empty directory.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.03G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

README.md: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

wav2vec2-large-xlsr-53/pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Missing tensor: mel_transformer.spectrogram.window
Missing tensor: mel_transformer.mel_scale.fb


In [ ]:
# 4. Run inference
device = torch.device("cuda")
model.to(device)
#"@W2WMoive" và ThanhPahm
input_text = "@W2WMoive: xxxxx"  # nhập giọng đọc cần và text cần chuyển đổi
prompt = f"<|task_tts|><|start_content|>{input_text}<|end_content|><|start_global_token|>"

inputs = tokenizer([prompt], return_tensors="pt").to(device)
generated_ids = model.generate(**inputs, max_new_tokens=64)

# 5. Parse tokens
output_text = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
semantic_tokens = re.findall(r"<\|bicodec_semantic_(\d+)\|>", output_text)
global_tokens = re.findall(r"<\|bicodec_global_(\d+)\|>", output_text)

# 6. Detokenize (CẦN audio_tokenizer)
audio_tokenizer.model.to(device)
wav = audio_tokenizer.detokenize(
    torch.tensor([int(t) for t in global_tokens]).unsqueeze(0).to(device),
    torch.tensor([int(t) for t in semantic_tokens]).unsqueeze(0).to(device)
)

# 7. Save audio
sf.write("output.wav", wav, 16000)

In [ ]:
from IPython.display import Audio, display

audio_file_path = "output.wav"
display(Audio(audio_file_path))